In [13]:
import numpy as np
import matplotlib.pyplot as plt
from obspy.core import read, Stream
import pandas as pd
from scipy.interpolate import interp1d
import random

In [9]:
sw4_path = '/Users/sydneydybing/SW4/strain/'
real_path = '/Users/sydneydybing/StrainProject/2024/'

In [10]:
path = '/Users/sydneydybing/SW4/strain/experiments/realistic_eq_sizes/'
stas = pd.read_csv(path + 'stas_w_p_s_traveltimes.csv')
hypo_lat = 35.7793
hypo_lon = -117.6204
hypo_depth = 7.5
stas

name    lon   lat  hypo_dist_km  p_arrival  s_arrival
0      E1 -118.5  35.0    117.735516  19.484985  33.835013
1      E2 -118.5  35.1    109.811069  18.258040  31.667682
2      E3 -118.5  35.2    102.476405  17.099509  29.636367
3      E4 -118.5  35.3     95.867056  16.055540  27.802686
4      E5 -118.5  35.4     90.142773  15.151369  26.214599
..    ...    ...   ...           ...        ...        ...
318  E319 -116.7  36.2     95.223479  15.953885  27.624136
319  E320 -116.7  36.3    101.083463  16.879489  29.249912
320  E321 -116.7  36.4    107.770552  17.935735  31.105169
321  E322 -116.7  36.5    115.140783  19.089840  33.125357
322  E323 -116.7  36.6    123.071535  20.297591  35.294405

[323 rows x 6 columns]

In [11]:
# # Event info

# # Read in event location file 
# event_file = 'M6_catalog_2004-2024.csv'
# eventLocs = pd.read_csv(real_path + event_file)

# # Pick your event
# event = eventLocs.iloc[11] # Ridgecrest mainshock

# hypo_lat = event.latitude
# hypo_lon = event.longitude
# hypo_depth = event.depth # km

In [25]:
stas_sub = ['E144', 'E126', 'E108', 'E90', 'E72', 'E54', 'E36', 'E18']
noise_stas = ['B072', 'B079', 'B082', 'B087', 'B916', 'B917', 'B918', 'B921']
Mws = [5, 6, 7, 8]
colors = ['blue', 'green', 'orange', 'red']

for idx in range(len(stas_sub)):

    sta = stas_sub[idx]

    i = np.where(stas.name.values == sta)[0]
    print(stas.name.values[i])
    hypdist = stas.hypo_dist_km.values[i]
    theo_p_arr = stas.p_arrival.values[i]
    theo_s_arr = stas.s_arrival.values[i]

    for idx2 in range(len(Mws)):
        
        Mw = Mws[idx2]
        color = colors[idx2]

        both_rms_strain = np.load(path + 'npys_for_mcmc/M' + str(Mw) + '/rms_strain/' + str(sta) + '.npy')
        time_sw4_strain = both_rms_strain[:,0]
        rms_sw4_strain = both_rms_strain[:,1]
        
        # Load the real noise
        noise_sta = random.choice(noise_stas)
        print('Noise station for M' + str(Mw) + ': ' + str(noise_sta))
        if noise_sta != 'B918':
            rms_real_noise_st = read(real_path + 'strain_data/noise/rms/2019-07-06_M7.1/' + str(noise_sta) + '.mseed')
        else:
            rms_real_noise_st = read(real_path + 'strain_data/noise/rms/2019-08-29_M6.3/' + str(noise_sta) + '.mseed')
            
        time_noise_strain = rms_real_noise_st[0].times()[:1201]
        rms_noise_strain = rms_real_noise_st[0].data[:1201]
        
        # print(max(time_sw4_strain))
        # print(max(time_noise_strain))
        
        # Interpolate SW4 strain to 20 Hz
        f = interp1d(time_sw4_strain, rms_sw4_strain)
        interp_rms_sw4_strain = f(time_noise_strain)
        
        # Add noise to SW4 strain
        noisy_sw4_rms_strain = interp_rms_sw4_strain + rms_noise_strain
        
        # Rename variables to match old processing
        time_strain = time_noise_strain
        rms_strain = interp_rms_sw4_strain
        noisy_rms_strain = noisy_sw4_rms_strain

        # Calculate peak strain - noisy rms and noisy filtered rms
        strain_p_i = np.abs(time_strain - theo_p_arr).argmin() # Get the index where the P-wave arrives

        mod_noisy_rms_strain = noisy_rms_strain.copy()
        noisy_p_arr_strain = noisy_rms_strain[strain_p_i]
        mod_noisy_rms_strain[:strain_p_i] = noisy_p_arr_strain # Strain before the P wave is set to be the same as the strain AT the P wave
        noisy_peak_strain = mod_noisy_rms_strain.copy()
        for k in range(0,len(mod_noisy_rms_strain)):
            if k == 0:
                strain = mod_noisy_rms_strain[0]
                max_strain = strain
            else:
                # Grab progressively longer windows and save the biggest strain
                strain = mod_noisy_rms_strain[:k+1] # Has to be k+1 because slicing doesn't include last one
                max_strain = max(strain)
            # Put peak strain back into the output stream
            noisy_peak_strain[k] = max_strain 

        ### Save the new data ###

        both_noisy_rms_strain = np.column_stack((time_strain, noisy_rms_strain))
        np.save(path + 'npys_for_mcmc/M' + str(Mw) + '/noisy_rms_strain/' + str(sta) + '.npy', both_noisy_rms_strain)

        both_noisy_peak_strain = np.column_stack((time_strain, noisy_peak_strain))
        np.save(path + 'npys_for_mcmc/M' + str(Mw) + '/noisy_peak_strain/' + str(sta) + '.npy', both_noisy_peak_strain)





['E144']
Noise station for M5: B921
Noise station for M6: B082
Noise station for M7: B087
Noise station for M8: B079
['E126']
Noise station for M5: B917
Noise station for M6: B917
Noise station for M7: B072
Noise station for M8: B072
['E108']
Noise station for M5: B082
Noise station for M6: B916
Noise station for M7: B918
Noise station for M8: B918
['E90']
Noise station for M5: B087
Noise station for M6: B072
Noise station for M7: B079
Noise station for M8: B082
['E72']
Noise station for M5: B917
Noise station for M6: B921
Noise station for M7: B072
Noise station for M8: B916
['E54']
Noise station for M5: B918
Noise station for M6: B917
Noise station for M7: B917
Noise station for M8: B921
['E36']
Noise station for M5: B072
Noise station for M6: B916
Noise station for M7: B921
Noise station for M8: B082
['E18']
Noise station for M5: B072
Noise station for M6: B917
Noise station for M7: B921
Noise station for M8: B072
